$$VC.KeyGen(1^{k}, q)$$
$$h_i = g^{z_i} \space h_{i,j,j\neq i}=g^{z_iz_j}$$

In [95]:
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, pair
import random
from Crypto.Random import random
import time
#mul == g**scalar
#sum = g + g2
#div = g/g2 = g-g2

"""
KeyGen requires:
    g : generator of group
    q: length of message list
"""
def keygen(g:G1,q:int) -> tuple:

    #list comprehension version
    
     #calculate zi element <-$ Zp
    z=[random.randint(0,q) for i in range(0,q)]

    #calculate hi as g^{zi}
    hi = [g**z[i] for i in range(len(z))]
    
    #calculate hij as g^{zi*zj}
    h_ij = [[g**(z[i]*z[j]) if i!=j else 0 for j in range(len(z))] for i in range(len(z))]

    return hi, h_ij, z

dim = 1000

group = PairingGroup('SS512')
g = group.random(G1)

    
messages = []

for i in range(0,dim):
    messages.append(random.randint(0,100))

start = time.time()
hi,h_ij,z = keygen(g, q = dim)
end = time.time()

print("Key gen required: " + str((end-start)*1000) + " ms")

Key gen required: 268.1436538696289 ms


$$VC.Com_{pp}(m1, . . . , m_q)$$
$$C = h_1^{m_1}h_2^{m_2}\cdot \cdot \cdot h_q^{m_q}$$

In [96]:
"""
Commit requires:
    messages: list of messages to commit
    hi: public parameter as g^{zi} for 1<=i<=q
"""
def commit(messages:list, hi:list) -> G1:
    #default value
    C = hi[0]**(0)
    for i in range(len(messages)):
        #calculate C as hi^{mi} * .. * hn^{mn}
        C = C+(hi[i]**(messages[i]))
    return C

start = time.time()
C = commit(messages,hi)
print(C)
end = time.time()

print("Commit required: " + str((end-start)*1000) + " ms")

[1534336064709618285996759901362965585029891456140589248117755678608719174722550031107830518085054167947509564012751261043110170870663212426757739094457461, 4342106904707316679283055632009110767920518398570194501935793348838263597342140833736284137485288941643419151495745964641654758920112402989635701274695557]
Commit required: 45.61424255371094 ms


$$VC.Open_{pp}(m_i, i, aux)$$
$$Λ_{i} = \prod_{j=1,j\neq i}^q h_{i,j}^{m_j} \space = \space (\prod_{j=1,j\neq i}^q h_j^{m_j})^{z_i}$$

In [97]:
"""
Open requires:
    i : index of message to open
    messages: list of messages
    h_ij: public parameter as g^{zi*zj}
    g: generator of group

"""
def open(i:int, messages:list, h_ij:list, g:G1) -> G1:
    #default value
    A = g**(0)
    for j in range(len(messages)):
        #skip i == j
        if(i==j):
            continue
        #calculate A as hij^{mij} * ... * hij^{mij}
        A = A+(h_ij[i][j]**(messages[j]))
    return A
start = time.time()
A = open(0,messages,h_ij,g)
print(A)
end = time.time()

print("Opening required: " + str((end-start)*1000) + " ms")

[741041910248802830447882257774084482100643133880763133803322271761351964202815291533083290375801630496218130809123445245301923123743752730960828653914710, 8322829934763621497478646788042434964310553370372742077691286619880632359516551201578913160366031946384831058691848019928742572558808062429511940954890310]
Opening required: 44.99626159667969 ms


$$VC.Open_{pp}(m_i, i, aux)$$
$$\space (\prod_{j=1,j\neq i}^q h_j^{m_j})^{z_i}$$

In [98]:
"""
Open2 is an alternative opening method which requires:
    messages: list of messages
    i : index of message to open
    hi: public parameter as g^{zi}
    z: public parameter as z1,...zq <-$ Zp
    g: generator of group
"""
def open2(messages:list,i:int,hi:list,z:list,g:G1) -> G1:
    #default value
    A = g**(0)
    for j in range(len(messages)):
        #skip i == j
        if(i==j):
            continue
        #calculate A as hi^{mj*zj}
        A = A+(hi[j]**(messages[j]*z[j]))
    return A

A2 = open2(messages,0,hi,z,g)
print(A2)

[2623550911971653002192644096961767626982741198812142171403117204173643690928590690794842491106320993887208520842130967488926080728474666131446535549719170, 7568555391772470192334389940682788149618874860744637166564943407966054389547658756653004435848164806984904578375462591768565293055587497395864879087252374]


$$VC.Ver_{pp}(C, m_i, i, Λ_i)$$
$$e(C/h_i^{m_i},h_i)=e(Λ_i,g)

In [99]:
"""
Verify requires:
    C: commitment value
    message: message to verify
    i: index of message in messages list
    A: proof of message at pos i
    g: generator of group
    hi: public parameter as g^{zi}
"""
def verify(C: G1, message:int, i:int, A:G1, g:G1, hi:list) -> bool:
    
    #calculate external divisor as hi^{mi}
    externsub = hi[i]**(message)
    #calculate first member of the first pairing as C/hi^{mi}
    C = C/(externsub)
    
    #calculate first pairing as e(C/hi^{mi},hi)
    t1 = pair(C,hi[i])
    #calculate second pairing as e(Ai,g)
    t2 = pair(A,g)
    return t1 == t2
start = time.time()
print("Verify output: " + str(verify(C,messages[0],0,A,g,hi)))
end = time.time()

print("Verifiy required: " + str((end-start)*1000) + " ms")

Verify output: True
Verifiy required: 1.6162395477294922 ms


$$VC.Update_{pp}C, m, m', i)$$
$$C' = C\cdot h_i^{m'-m}$$

In [100]:
"""
Update commitment requires:
    C: commitment value
    oldmessage: the old message 
    newmessage: the new message to commit
    i: index of previous and new message
    hi: public parameter as g^{zi}
"""
def update(C:G1, oldmessage:int, newmessage:int,i:int,hi:list)-> G1:
    #calculate the new commitment C' as C*hi^{m'-m}
    newc = C+hi[i]**(newmessage-oldmessage)
    return newc
start = time.time()
newc = update(C,messages[0],10000002,0,hi)

end = time.time()

print("Update commitment required: " + str((end-start)*1000) + " ms")
verified2 = verify(newc,10000002,0,A,g,hi)
if(verified2):
    print("Verifying update worked")
else:
    print("Verifying update not worked")

Update commitment required: 0.21839141845703125 ms
Verifying update worked


$$VC.ProofUpdate_{pp}(C, Λ_j, m', U)$$
$$Λ_j' = Λ_j \cdot (h_i^{m'-m})^{z_j} = Λ_j \cdot h_{i,j}^{m'-m}$$

In [101]:
"""
Update proof requires:
    A: old proof
    newmessage: new message to commit
    oldmessage: old message
    i: index of old message
    j: index of new message
    h_ij: public parameter as g^{zi*zj}
"""
def updateProof(A:G1,newmessage:int,oldmessage:int,i:int,j:int,h_ij:list) -> G1:
    #calculate newproof Aj' as Aj*(hij^{m'-m})
    newproof = A+(h_ij[j][i]**(newmessage - oldmessage))
    return newproof

oldproof = open(1,messages,h_ij,g)
start = time.time()
newproof = updateProof(oldproof,10000002,messages[0],0,1,h_ij)

end = time.time()

print("Update proof required: " + str((end-start)*1000) + " ms")

verified3 = verify(newc,messages[1],1,newproof,g,hi)

if(verified3):
    print("Verifying update proof worked")
else:
    print("Verifying update proof not worked")

Update proof required: 0.2932548522949219 ms
Verifying update proof worked
